# Import necessary packages


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
import joblib


# Load Data

In [2]:
processed_fraud_data = pd.read_csv('../data/processed/processed_fraud_data.csv')
creditcard_data = pd.read_csv('../data/raw/creditcard.csv')


In [3]:
creditcard_data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Convert datetime strings to datetime objects
processed_fraud_data['signup_time'] = pd.to_datetime(processed_fraud_data['signup_time'])
processed_fraud_data['purchase_time'] = pd.to_datetime(processed_fraud_data['purchase_time'])

# Extract useful datetime components
processed_fraud_data['signup_hour'] = processed_fraud_data['signup_time'].dt.hour
processed_fraud_data['signup_day'] = processed_fraud_data['signup_time'].dt.dayofweek
processed_fraud_data['purchase_hour'] = processed_fraud_data['purchase_time'].dt.hour
processed_fraud_data['purchase_day'] = processed_fraud_data['purchase_time'].dt.dayofweek

# Drop the original datetime columns
# processed_fraud_data = processed_fraud_data.drop(columns=['signup_time', 'purchase_time'])

In [5]:
# Set up MLflow tracking URI and experiment
mlflow.set_tracking_uri("file:///mlruns")  # Local tracking
mlflow.set_experiment("Fraud Detection Models")

<Experiment: artifact_location='file:c:/mlruns/657951383817075237', creation_time=1729513891557, experiment_id='657951383817075237', last_update_time=1729513891557, lifecycle_stage='active', name='Fraud Detection Models', tags={}>

# Step 1: Data Preparation
## Separate Features and Target Variables

In [6]:
# For fraud_data
fraud_X = processed_fraud_data.drop(columns=['class'])
fraud_y = processed_fraud_data['class']
# For creditcard_data
creditcard_X = creditcard_data.drop(columns=['Class'])
creditcard_y = creditcard_data['Class']

In [7]:
# Split fraud_data
fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test = train_test_split(fraud_X, fraud_y, test_size=0.3, random_state=42)

# Split creditcard_data
creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test = train_test_split(creditcard_X, creditcard_y, test_size=0.3, random_state=42)
     

In [8]:
# Define preprocessing for numerical and categorical features
numeric_features = ['purchase_value', 'age']  # Example numeric features
categorical_features = ['source', 'browser', 'sex','signup_hour','signup_day', 'purchase_hour', 'purchase_day']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Step 2: Model Selection and Training
### Logistic Regression, Decision Tree, Random Forest, Gradient Boosting

In [9]:
# Model Selection
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier()
}

In [10]:

# Function to train and evaluate models
def train_evaluate_model_fraud(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(pipeline, model_name)

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [11]:
# Function to train and evaluate models
def train_evaluate_model_credit(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(pipeline, model_name)

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [12]:

# Train and evaluate models on fraud data
fraud_results = []
for model_name, model in models.items():
    fraud_results.append(train_evaluate_model_fraud(model_name, model, fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test))


c:\Users\habteyes.asfaw\10Accadamy\fintech-fraud-detection\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/10/26 23:19:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/26 23:19:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/26 23:21:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/26 23:22:10 WARNING mlflow.models.model: Model logged

In [13]:
# Train and evaluate models on credit card data
creditcard_results = []
for model_name, model in models.items():
    creditcard_results.append(train_evaluate_model_credit(model_name, model, creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test))
     

c:\Users\habteyes.asfaw\10Accadamy\fintech-fraud-detection\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/10/26 23:26:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/26 23:26:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/26 23:29:49 WARNING mlflow.models.model: Model 

In [14]:
 # Display results
fraud_results_df = pd.DataFrame(fraud_results)
creditcard_results_df = pd.DataFrame(creditcard_results)

print("Fraud Data Results:\n", fraud_results_df)
print("Credit Card Data Results:\n", creditcard_results_df)

Fraud Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.903934   0.000000  0.000000  0.000000
1        Decision Tree  0.907599   0.517410  0.566900  0.541026
2        Random Forest  0.955374   0.996512  0.537346  0.698202
3    Gradient Boosting  0.906592   0.899225  0.031166  0.060244
4                  MLP  0.944456   0.819106  0.541376  0.651893
Credit Card Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.998654   0.575540  0.588235  0.581818
1        Decision Tree  0.999146   0.700637  0.808824  0.750853
2        Random Forest  0.999625   0.956140  0.801471  0.872000
3    Gradient Boosting  0.998584   0.894737  0.125000  0.219355
4                  MLP  0.997858   0.412639  0.816176  0.548148


In [15]:
# Save the model to a file
joblib.dump(fraud_results, '../models/fraud_model.pkl')
joblib.dump(creditcard_results, '../models/creditcard_model.pkl')

print("Model saved as 'models/fraud_model and creditcard_model.pkl'")


Model saved as 'models/fraud_model and creditcard_model.pkl'
